# API

## Libreria Requests

In [177]:
%pip install requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [178]:
import requests

## Datos API

In [179]:
URL = 'http://localhost:3000'
# URL =  "https://raphael.chan.ing.puc.cl/iic2233-AY12"

## Consultas

### GET

La request `GET` se utiliza cuando solo se quiere obtener información desde la URL. En la mayoría de los casos estas consultas no llevan un body, pero pueden tener tanto parametros de query `majors/q=major` o path parameters `majors/1`.

In [180]:
resp = requests.get(f'{URL}/major/software')
print(resp.json())

{'nombre': 'Ingeniería de Software', 'requisitos': {'relacion': 'AND', 'subrequisitos': [{'nombre': 'Exploratorio', 'relacion': 'OR', 'subrequisitos': ['AQH0000', 'ICC2304', 'ICE2006', {'relacion': 'OR', 'subrequisitos': ['ICE2623', 'IMM2003']}, 'ICH1005', 'IBM1005', 'ICH1104', 'ICH2304', 'ICM1001', {'relacion': 'OR', 'subrequisitos': ['ICS1113', 'ICS113H']}, 'ICT2904', 'IEE1100', 'IIC1005', 'IIQ1001', 'IIQ2663', 'IMM1003', 'IMT1001', 'IDI1015', 'ING1024', {'relacion': 'OR', 'subrequisitos': ['IRB1001', 'IRB2001']}, 'IFI1001', 'ICC1001']}, {'nombre': 'Minimos de Major', 'relacion': 'AND', 'subrequisitos': ['IIC1253', 'IIC2113', 'IIC2133', 'IIC2143', 'IIC2173', 'IIC2233', 'IIC2413', 'IIC2513', 'IIC2713', 'IIC2154']}]}}


### POST

La request `POST` se utiliza cuando se quiere enviar información para que la aplicación modifique algo o utilice dicha información de alguna manera. Estas requests se deben enviar con un body, el cual puede ser un json.

In [181]:
# Si eres ayudante, descomenta esto
with open('private_key.txt', mode='r') as secret_key_file:
    token = secret_key_file.read().strip()

# Si eres ayudante, descomenta esto
# token = '' # Ingresa tu numero de alumno

In [182]:
headers = {
    'Authorization' : f"Bearer {token}"
}

body = {
    "cursos_aprobados" : [
        "IIC1253",
        "IIC2233",
        "IIC2133",
        "IIC2143",
        "IIC2413",
        "IIC2513",
        "IIC2154",
        "AQH0000",
        "IIC2304",
        "ICE2006",
        "ICE2623",
        "IIC1005",
        "IIC1006",
        "IIC2343",
        "IIC2333",
        "IIC2613"
    ]
}

resp = requests.post(f'{URL}/major/compu/validate', json=body, headers=headers)
print(resp.json())

{'major': 'compu', 'aprobado': False}


Ademas nuestra API posee una `POST` request para poder agregar mas tokens a un usuario  

In [183]:
headers = {
    'Authorization' : f"Bearer {token}"
}

body = {
    "numero_alumno" : token,
    "tokens" : 12
}

resp = requests.post(f'{URL}/tokens/add', json=body, headers=headers)
print(resp.json())

{'numero_alumno': '17640040', 'tokens': 15, 'warning': 'Remember that the maximum amount of tokens is 15. If you want more tokens, email lfvansintjan@uc.cl'}


### PUT

La request `PUT` se utiliza cuando se quiere actualizar información existente o, crear nueva informacion en el caso de que no esta no haya existido en la base de datos de la aplicación. Estas requests se deben enviar con un body, el cual puede ser un json.

In [184]:
headers = {
    'Authorization' : f"Bearer {token}"
}

# Forma segura de obtenerlo
major_name = f"Major del alumno {token.split('+')[0]}"

body = {
    "nombre": major_name,
    "requisitos": {
        "relacion": "AND",
        "subrequisitos": [
            {
                "nombre": "Pack 1",
                "relacion": "AND",
                "subrequisitos": [
                    "IIC2333",
                    "ICE2164",
                    "AFE4326",
                ]
            },
            {
                "nombre": "Pack 2",
                "relacion": "AND",
                "subrequisitos": [
                    "IIC3257",
                    "IGM6547",
                    "IIC6424"
                ]
            },
            {
                "nombre": "Track",
                "relacion": "OR",
                "subrequisitos": [
                    {
                        "nombre": "Track 1",
                        "relacion": "AND",
                        "subrequisitos": [
                            "IIC2343",
                            "IIC2613"
                        ]
                    },
                    {
                        "nombre": "Track 2",
                        "relacion": "AND",
                        "subrequisitos": [
                            "IIC2713",
                            "IIC2733",
                        ]
                    }
                ]
            }
        ]
    }
}

resp = requests.put(f'{URL}/major/{major_name}', json=body, headers=headers)
print(resp.json())

{'message': "Major 'Major del alumno 17640040' updated successfully"}


### DELETE

La request `DELETE` se utiliza cuando se quiere eliminar información de la aplicación. Estas requests por lo general utilizan path params para identificar el objeto que se quiere eliminar `majors/23`.

In [185]:
headers = {
    'Authorization' : f"Bearer {token}"
}

resp = requests.delete(f'{URL}/major/compu/AQH0000', json=body, headers=headers)
print(resp.json())

{'message': "Course AQH0000 from Major 'compu' deleted successfully"}


# RegEx

Las expresiones regulares (o RegEx por su sigla en inglés) nos ayudan a encontrar patrones dentro de *strings* para así poder filtrar los que necesitemos.

## Funciones necesarias

En esta parte definimos las librerías necesarias y funciones que nos ayudaran en los ejemplos que mostraremos

In [186]:
import re

In [187]:

def list_desempac(my_list, iterable):
    for char in iterable:
        for elem in char:
            my_list.append(elem)

headers = {
    'Authorization' : f"Bearer {token}"
}

resp = requests.get(f'{URL}/major/software/packages', headers=headers)

data = resp.json()
packages = filter(lambda val: isinstance(val, dict), list(data.values())[1])

all_courses = []
[list_desempac(all_courses, char['courses'].values()) if char.get('courses') else [list_desempac(all_courses, elem["courses"].values()) for elem in char['packages']] for char in packages]
print(all_courses)

['AQH0000', 'ICC2304', 'ICE2006', 'ICE2623', 'IMM2003', 'ICH1005', 'IBM1005', 'ICH1104', 'ICH2304', 'ICM1001', 'ICS1113', 'ICS113H', 'ICT2904', 'IEE1100', 'IIC1005', 'IIQ1001', 'IIQ2663', 'IMM1003', 'IMT1001', 'IDI1015', 'ING1024', 'IRB1001', 'IRB2001', 'IFI1001', 'ICC1001', 'IIC1253', 'IIC2113', 'IIC2133', 'IIC2143', 'IIC2173', 'IIC2233', 'IIC2413', 'IIC2513', 'IIC2713', 'IIC2154']


## Consultas

### Ejemplo con `match`
Buscamos todos los cursos que son del DCC (comienzan con IIC)

In [188]:
doct_courses = filter(lambda course: re.match(r'^IIC.*', course), all_courses)
[char for char in doct_courses]

['IIC1005',
 'IIC1253',
 'IIC2113',
 'IIC2133',
 'IIC2143',
 'IIC2173',
 'IIC2233',
 'IIC2413',
 'IIC2513',
 'IIC2713',
 'IIC2154']

Analicemos la búsqueda RegEx de `^IIC.*` 🔬

El `^` especifica que lo que buscamos está **al principio** del string, en este caso, queremos buscar algo que **parta** con `IIC`

Luego, los caracteres `.*` indican que lo siguiente que buscamos puede ser **cualquier cosa** y de **cualquier longitud**.

### Ejemplo con `fullmatch`

Ahora, imaginemos que todas las siglas de los cursos primero contienen 3 letras y luego 4 números (formato ABC1234), por lo que queremos filtrar los valores en que esto no pasa.

Usamos `fullmatch()` para esto, ya que queremos que el **string completo** cumpla con este patrón, y no solo un *substring*. En nuestro ejemplo, recibimos una lista de 35 cursos a través de nuestra consulta, y buscamos cuántos cumplen con el formato indicado

In [189]:
course_syntax = filter(lambda course: re.fullmatch(r'^[A-Z]+[0-9]{4}', course), all_courses)
cursos_validos = [char for char in course_syntax]
print(len(cursos_validos))

34


El resultado es 34 porque el curso que no cumple exactamente con el formato es ICS113H, el amado y odiado Opti Honors 📚, ya que contiene una H al final en vez de un número.

### Ejemplo con `search`

Esta función funciona de manera similar a `match`, con la diferencia de que no necesariamente busca la expresión regular **al inicio** del *string*.

Pediremos todos los cursos que sean "introductorios", lo que definimos como que la parte numérica de su sigla sea un valor 1000 y algo.

In [190]:
intro_courses = filter(lambda course: re.search(r'1[0-9]{3}', course), all_courses)
[char for char in intro_courses]

['ICH1005',
 'IBM1005',
 'ICH1104',
 'ICM1001',
 'ICS1113',
 'IEE1100',
 'IIC1005',
 'IIQ1001',
 'IMM1003',
 'IMT1001',
 'IDI1015',
 'ING1024',
 'IRB1001',
 'IFI1001',
 'ICC1001',
 'IIC1253']

Como pueden ver, el filtro solo buscaba un patrón numérico, que no necesariamente estaba al inicio del string (de hecho, no lo está en ningun caso). Si ese mismo filtro se hiciera con `match` o `fullmatch`, retornaría vació. Si quieres puedes probarlo por tu cuenta 😉.

### Ejemplo `sub`

La función `sub` busca cierto patrón y lo **reemplaza** con otro indicado. Puede ser útil para cambiar formatos directamente, o eliminar caracteres indeseados (reemplazando un valor con `''`)

Eliminaron LET del plan común e Ingeniería termina con serios problemas de ortografía, al punto que renombran la Escuela como "Eskuela de Hingeniería" 🥴, por lo que tendremos que reemplazar todas las siglas que partían con I por H.

In [191]:
lowercase_courses = map(lambda course: re.sub(r'^I{1}', r'H', course), all_courses)
[char for char in lowercase_courses]

['AQH0000',
 'HCC2304',
 'HCE2006',
 'HCE2623',
 'HMM2003',
 'HCH1005',
 'HBM1005',
 'HCH1104',
 'HCH2304',
 'HCM1001',
 'HCS1113',
 'HCS113H',
 'HCT2904',
 'HEE1100',
 'HIC1005',
 'HIQ1001',
 'HIQ2663',
 'HMM1003',
 'HMT1001',
 'HDI1015',
 'HNG1024',
 'HRB1001',
 'HRB2001',
 'HFI1001',
 'HCC1001',
 'HIC1253',
 'HIC2113',
 'HIC2133',
 'HIC2143',
 'HIC2173',
 'HIC2233',
 'HIC2413',
 'HIC2513',
 'HIC2713',
 'HIC2154']

### Ejemplo `split`

La función `split` tiene la intención de **separar** en 2 o más elementos un string, usando como **separador** el patrón que se le de. Para este caso, el ejemplo más obvio es separar la parte numérica de una sigla con la alfabética.

Solo para hacerlo más entretenido, agruparemos después las siglas por las "letras" del principio, es decir, agruparemos los ramos por la facultad que los imparte.

In [192]:
split_parts = map(lambda course: re.split(r'(\b\w{3})(\w+)', course), all_courses)
ramos_por_facultad = [chars[1:-1] for chars in split_parts] 
# chars por si solo da 2 elementos vacíos al principio y al final, los cuales se eliminan 
ramos_por_facultad

[['AQH', '0000'],
 ['ICC', '2304'],
 ['ICE', '2006'],
 ['ICE', '2623'],
 ['IMM', '2003'],
 ['ICH', '1005'],
 ['IBM', '1005'],
 ['ICH', '1104'],
 ['ICH', '2304'],
 ['ICM', '1001'],
 ['ICS', '1113'],
 ['ICS', '113H'],
 ['ICT', '2904'],
 ['IEE', '1100'],
 ['IIC', '1005'],
 ['IIQ', '1001'],
 ['IIQ', '2663'],
 ['IMM', '1003'],
 ['IMT', '1001'],
 ['IDI', '1015'],
 ['ING', '1024'],
 ['IRB', '1001'],
 ['IRB', '2001'],
 ['IFI', '1001'],
 ['ICC', '1001'],
 ['IIC', '1253'],
 ['IIC', '2113'],
 ['IIC', '2133'],
 ['IIC', '2143'],
 ['IIC', '2173'],
 ['IIC', '2233'],
 ['IIC', '2413'],
 ['IIC', '2513'],
 ['IIC', '2713'],
 ['IIC', '2154']]

In [193]:
from collections import defaultdict
facultades = defaultdict(list)

for sublista in ramos_por_facultad:
    letras = sublista[0]
    numeros = sublista[1]

    facultades[letras].append(numeros)

facultades


defaultdict(list,
            {'AQH': ['0000'],
             'ICC': ['2304', '1001'],
             'ICE': ['2006', '2623'],
             'IMM': ['2003', '1003'],
             'ICH': ['1005', '1104', '2304'],
             'IBM': ['1005'],
             'ICM': ['1001'],
             'ICS': ['1113', '113H'],
             'ICT': ['2904'],
             'IEE': ['1100'],
             'IIC': ['1005',
              '1253',
              '2113',
              '2133',
              '2143',
              '2173',
              '2233',
              '2413',
              '2513',
              '2713',
              '2154'],
             'IIQ': ['1001', '2663'],
             'IMT': ['1001'],
             'IDI': ['1015'],
             'ING': ['1024'],
             'IRB': ['1001', '2001'],
             'IFI': ['1001']})